# Week 3 Assignment #

## Create Toronto data frame and Submit a link to your Notebook on your Github repository. (10 marks) ##

In [1]:
# import necessary packages
import pandas as pd

**First read the table from wiki**

In [2]:
# wikipedia url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

**Extract the table**

**Read html table into pandas dataframe**

In [3]:
# Read html table into pandas dataframe
my_list = pd.read_html(url, header=0,match='Postcode')
df = my_list[0]

In [4]:
# drop the rows with borough not assigned
df = df[df['Borough']!='Not assigned']

In [5]:
# sort data drame by postal codes
df=df.sort_values(by=['Postcode'])

In [6]:
# initialize another dataframe with first row of the table sorted by postal code
df2 = pd.DataFrame(df.iloc[0]).transpose()

In [7]:
# loop through the sorted table
j = 1
for i in range(1, df.shape[0]):
    # if the postal code is same as the last postal code, append the neighbourhood
    if df.iloc[i]['Postcode'] == df2.iloc[j-1]['Postcode']:
        df2.iloc[j-1]['Neighbourhood'] = df2.iloc[j-1]['Neighbourhood'] + ', ' + df.iloc[i]['Neighbourhood']
    # otherwise add the row to the new dataframe
    else:
        df2 = df2.append(pd.DataFrame(df.iloc[i]).transpose())
        j = j + 1 

In [8]:
# if the neighbourhood is not assigned, assigned the borough value in the same row to neighbourhood
df2.loc[df2['Neighbourhood'] == 'Not assigned','Neighbourhood']=df2.loc[df2['Neighbourhood'] == 'Not assigned','Borough']

In [9]:
df2.shape

(103, 3)

## Add geocode info and submit a link to the new Notebook on your Github repository. (2 marks) ##

In [10]:
# Read the geocode data
geocode_data = pd.read_csv('http://cocl.us/Geospatial_data')
print('geocode data read')

geocode data read


In [11]:
# Join the data frames to add the latitude and longitude by postal code
df3 = pd.merge(df2, geocode_data, how='left', left_on=['Postcode'], right_on=['Postal Code'])
df3.drop(['Postal Code'], axis=1)
df3.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


## Explore and cluster the neighborhoods in Toronto. (3 marks) ##

**Install and import the libraries**

In [12]:
# Install the libraries
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes


# Import the librarries
import numpy as np # library to handle data in a vectorized manner

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  21.42 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  35.70 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  54.25 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  21.23 MB/s
vincent-0.4.4- 100% |###################

**Create the map of TO**

In [13]:
TO_boroughs_data = df3[df3['Borough'].str.contains("Toronto")].reset_index(drop=True)
TO_boroughs_data.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572
3,M4M,East Toronto,Studio District,M4M,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790


In [14]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
# create map of Toronto using latitude and longitude values
map_TO = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(TO_boroughs_data['Latitude'], TO_boroughs_data['Longitude'], TO_boroughs_data['Borough'], TO_boroughs_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TO)  
    
map_TO

**Define Foursquare Credentials and Version**

In [16]:
# The code was removed by Watson Studio for sharing.

**Explore the neighbourhoods of TO** 

 **Create a function to repeat the same process to all the neighborhoods in TO**

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
TO_venues = getNearbyVenues(names=TO_boroughs_data['Neighbourhood'],
                                   latitudes=TO_boroughs_data['Latitude'],
                                   longitudes=TO_boroughs_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
South Hill, Rathnelly, Forest Hill SE, Deer Park, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Regent Park, Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, Richmond, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill West, Forest Hill North
North Midtown, The Annex, Yorkville
Harbord, University of Toronto
Kensington Market, Grange Park, Chinatown
King and Spadina, CN Tower, South Niagara, Bathurst Quay, Island airport, Railway Lands, Harbourfront West
Stn A PO Boxes 25 The Esplanade
Underground city, First Canadian Place
Christie
Dovercourt Village, Dufferin
Trinity, Little Portugal
Exhibition Place, Brockton, Parkdale Village
High Park, The 

In [19]:
print(TO_venues.shape)
TO_venues.head()

(1695, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [20]:
print('There are {} uniques categories.'.format(len(TO_venues['Venue Category'].unique())))

There are 232 uniques categories.


In [21]:
TO_venues['cnt']=TO_venues.groupby(['Venue Category'])['Venue Category'].transform('count')

**Convert the venue category values to columns** 

In [22]:
# one hot encoding
TO_onehot = pd.get_dummies(TO_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
TO_onehot['Neighborhood'] = TO_venues['Neighborhood'] 

In [23]:
TO_onehot.shape

(1695, 232)

In [24]:
TO_grouped = TO_onehot.groupby('Neighborhood').mean().reset_index()
TO_grouped.shape

(38, 232)

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = TO_grouped['Neighborhood']

for ind in np.arange(TO_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TO_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, Richmond, King",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Asian Restaurant,Gym,Clothing Store,Bakery,Bar
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Café,Seafood Restaurant,Bakery,Italian Restaurant,Steakhouse,Farmers Market,Cheese Shop
2,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Park,Comic Shop,Recording Studio
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Bakery,Indian Restaurant,Pizza Place,Pub,Italian Restaurant,Pet Store,Jewelry Store
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Ice Cream Shop,Burger Joint,Bar,Falafel Restaurant,Indian Restaurant,Japanese Restaurant,Salad Place
5,Christie,Café,Grocery Store,Park,Baby Store,Nightclub,Convenience Store,Diner,Athletics & Sports,Coffee Shop,Italian Restaurant
6,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Burger Joint,Restaurant,Mediterranean Restaurant,Pub,Gastropub,Bubble Tea Shop
7,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Seafood Restaurant,Gym,Bakery,Steakhouse
8,Davisville,Sandwich Place,Dessert Shop,Coffee Shop,Seafood Restaurant,Italian Restaurant,Café,Sushi Restaurant,Pizza Place,Pharmacy,Burger Joint
9,Davisville North,Food & Drink Shop,Pizza Place,Grocery Store,Breakfast Spot,Park,Hotel,Sandwich Place,Clothing Store,Burger Joint,Yoga Studio


In [27]:
# set number of clusters
kclusters = 5

TO_grouped_clustering = TO_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TO_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 0, 0, 2, 2], dtype=int32)

In [28]:
type(kmeans)

sklearn.cluster.k_means_.KMeans

In [29]:
kmeans.cluster_centers_

array([[ 0.00043706,  0.00043706,  0.        , ...,  0.00043706,
         0.00076923,  0.00327513],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.01041667, ...,  0.        ,
         0.        ,  0.01686508],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [36]:
TO_merged = TO_boroughs_data

# add clustering labels
TO_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
TO_merged = pd.merge(TO_merged, neighborhoods_venues_sorted, left_on=['Neighbourhood'], right_on=['Neighborhood'])

TO_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031,0,The Beaches,Asian Restaurant,Coffee Shop,Pub,Yoga Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188,0,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Grocery Store,Brewery,Bubble Tea Shop,Caribbean Restaurant,Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572,2,"The Beaches West, India Bazaar",Park,Gym,Italian Restaurant,Pet Store,Pizza Place,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint
3,M4M,East Toronto,Studio District,M4M,43.659526,-79.340923,0,Studio District,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Music Store,Yoga Studio,Park,Seafood Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790,0,Lawrence Park,Park,Lake,Swim School,Dim Sum Restaurant,Bus Line,Yoga Studio,Doner Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(TO_merged['Latitude'], TO_merged['Longitude'], TO_merged['Neighborhood'], TO_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters